In [1]:
# -*- coding: utf-8 -*-
train_data_txt = "../training/msr_training.txt"
init_train_data = []
with open(train_data_txt,encoding='gbk') as trt:
    init_train_data = trt.readlines()
all_data = "".join(init_train_data)
train_data_split = [line.strip().split() for line in init_train_data]

print(train_data_split[0:5])
print(len(train_data_split))

[['“', '人们', '常', '说', '生活', '是', '一', '部', '教科书', '，', '而', '血', '与', '火', '的', '战争', '更', '是', '不可多得', '的', '教科书', '，', '她', '确实', '是', '名副其实', '的', '‘', '我', '的', '大学', '’', '。'], ['“', '心', '静', '渐', '知', '春', '似', '海', '，', '花', '深', '每', '觉', '影', '生', '香', '。'], ['“', '吃', '屎', '的', '东西', '，', '连', '一', '捆', '麦', '也', '铡', '不', '动', '呀', '？'], ['他', '“', '严格要求', '自己', '，', '从', '一个', '科举', '出身', '的', '进士', '成为', '一个', '伟大', '的', '民主主义', '者', '，', '进而', '成为', '一', '位', '杰出', '的', '党外', '共产主义', '战士', '，', '献身', '于', '崇高', '的', '共产主义', '事业', '。'], ['“', '征', '而', '未', '用', '的', '耕地', '和', '有', '收益', '的', '土地', '，', '不准', '荒芜', '。']]
86924


In [2]:
# 读取字符词典文件
char_vocab_path = "../training/char_vocabs.txt"
special_words = ['<PAD>', '<UNK>'] # 特殊词表示
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs
print(char_vocabs[0:10])

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

['<PAD>', '<UNK>', '!', '#', '%', '&', "'", '(', ')', '*']


In [3]:
# "BMES"标记的标签
label2idx = {"S": 0,
             "B": 1, 
             "M": 2,
             "E": 3
             }
# 索引和BMES标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

In [4]:
# 读取data
def data_list(path, vocab2idx, label2idx):
    with open(train_data_txt,encoding='gbk') as trt:
        lines = trt.readlines()
    datas, labels= [], []
    for line in lines: # line是一个字符串
        mline = line.strip().split() #mline是个词列表
        sent, tag = [],[]
        for i,word in enumerate(mline):
            if len(word)==1:
                sent.append(word)
                tag.append("S")
            else:
                s_word = ["M" for i in word]
                s_word[0] = "B"
                s_word[-1] = "E"
                sent+=list(word)
                tag+=s_word

        sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent]
        tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag]
        datas.append(sent_ids)
        labels.append(tag_ids)

    return datas, labels

In [5]:
datas, labels = data_list(train_data_txt, vocab2idx, label2idx)
print(len(datas),len(labels))
print(datas[5])
print([idx2vocab[idx] for idx in datas[5]])
print(labels[5])
print([idx2label[idx] for idx in labels[5]])

86924 86924
[122, 5800, 6506, 563, 2572, 240, 3007, 2530, 6209, 6440, 6802, 3007, 2530, 3903, 2102, 2065, 6209, 6440, 182]
['“', '这', '首', '先', '是', '个', '民', '族', '问', '题', '，', '民', '族', '的', '感', '情', '问', '题', '。']
[0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0]
['S', 'S', 'B', 'E', 'S', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'S']


In [6]:
snum = 70000
train_datas = datas[0:snum]
train_labels = labels[0:snum]
test_datas = datas[snum:]
test_labels = labels[snum:]
print(train_labels[0:10])

[[0, 1, 3, 0, 0, 1, 3, 0, 0, 0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 2, 2, 3, 0, 1, 2, 3, 0, 0, 1, 3, 0, 1, 2, 2, 3, 0, 0, 0, 0, 1, 3, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 2, 2, 3, 1, 3, 0, 0, 1, 3, 1, 3, 1, 3, 0, 1, 3, 1, 3, 1, 3, 1, 3, 0, 1, 2, 2, 3, 0, 0, 1, 3, 1, 3, 0, 0, 1, 3, 0, 1, 3, 1, 2, 2, 3, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 2, 2, 3, 1, 3, 0], [0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0], [0, 0, 0, 0, 1, 3, 1, 2, 3, 0, 0, 1, 2, 3, 0, 0, 1, 3, 0], [0, 1, 3, 1, 3, 1, 3, 0, 1, 2, 3, 1, 3, 0, 1, 2, 3, 1, 2, 2, 3, 0, 1, 3, 0, 1, 3, 0], [0, 1, 3, 1, 3, 0, 0, 1, 3, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 1, 3, 0, 0], [0, 0, 1, 3, 0, 1, 3, 1, 3, 0, 1, 3, 0, 0, 0, 0]]


In [7]:
from keras.utils import to_categorical

batch_size = 128

import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()
from crf_keras import CRF
from keras.layers import Dense, Embedding, Conv1D, Input
from keras.models import Model # 这里我们学习使用Model型的模型
import keras.backend as K # 引入Keras后端来自定义loss，注意Keras模型内的一切运算
                          # 必须要通过Keras后端完成，比如取对数要用K.log不能用np.log

embedding_size = 128
EPOCHS = 2
MAX_LEN=100
CLASS_NUMS=4

train_crf = False  # true train CRF-transaction Matrix

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
test_datas = sequence.pad_sequences(test_datas, maxlen=MAX_LEN)
test_labels = sequence.pad_sequences(test_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', test_datas.shape)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)#.reshape((70000,100, 5))
test_labels = keras.utils.to_categorical(test_labels, CLASS_NUMS)#.reshape((16924,100, 5))
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)



sequence = Input(shape=(None,), dtype='int32') # 建立输入层，输入长度设为None
embedding = Embedding(len(char_vocabs)+1,
                      embedding_size,
                     )(sequence) # 去掉了mask_zero=True
cnn = Conv1D(128, 3, activation='relu', padding='same')(embedding)
cnn = Conv1D(128, 3, activation='relu', padding='same')(cnn)
cnn = Conv1D(128, 3, activation='relu', padding='same')(cnn) # 层叠了3层CNN

if train_crf:
    crf = CRF(True) # 定义crf层，参数为True，自动mask掉最后一个标签
tag_score = Dense(CLASS_NUMS)(cnn) # 变成了5分类，第五个标签用来mask掉
if train_crf:
    tag_score = crf(tag_score) # 包装一下原来的tag_score
else:
    tag_score = Activation('softmax')(tag_score)

model = Model(inputs=sequence, outputs=tag_score)
model.summary()

if train_crf == False:
    model.compile(loss="categorical_crossentropy", # 用crf自带的loss
              optimizer='adam',
              metrics=["accuracy"] # 用crf自带的accuracy
             )
else:
    model.compile(loss=crf.loss, # 用crf自带的loss
                  optimizer='adam',
                  metrics=[crf.accuracy] # 用crf自带的accuracy
                 )
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)


model.save_weights("../model/CNN_seg_weight.h5")
model.save("../model/CNN_seg.h5")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

padding sequences
x_train shape: (70000, 100)
x_test shape: (16924, 100)
trainlabels shape: (70000, 100)
testlabels shape: (16924, 100)


W0825 19:31:41.651870 139756498421568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0825 19:31:41.658265 139756498421568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0825 19:31:41.797614 139756498421568 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



trainlabels shape: (70000, 100, 4)
testlabels shape: (16924, 100, 4)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         880000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         49280     
_________________________________________________________________
dense_1 (Dense)              (None, None, 4)           516       
________________________________________________________

W0825 19:31:41.972900 139756498421568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 63000 samples, validate on 7000 samples
Epoch 1/2
63000/63000 [==============================] - 62s 992us/step - loss: 0.1337 - acc: 0.9520 - val_loss: 0.0817 - val_acc: 0.9715
Epoch 2/2
63000/63000 [==============================] - 62s 978us/step - loss: 0.0643 - acc: 0.9776 - val_loss: 0.0664 - val_acc: 0.9769


In [8]:
BATCH_SIZE = 64
score = model.evaluate(test_datas, test_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

16924/16924 [==============================] - 3s 190us/step
['loss', 'acc']
[0.07004244630322956, 0.97601512709117]


In [35]:
def max_in_dict(d): # 定义一个求字典中最大值的函数
    key,value = list(d.items())[0]
    for i,j in list(d.items())[1:]:
        if j > value:
            key,value = i,j
    return key,value

def viterbi(nodes, trans): # viterbi算法，跟前面的HMM一致
    paths = nodes[0] # 初始化起始路径
    for l in range(1, len(nodes)): # 遍历后面的节点
        paths_old,paths = paths,{}
        for n,ns in nodes[l].items(): # 当前时刻的所有节点
            max_path,max_score = '',-1e10
            for p,ps in paths_old.items(): # 截止至前一时刻的最优路径集合
                score = ns + ps + trans[p[-1]+n] # 计算新分数
                if score > max_score: # 如果新分数大于已有的最大分
                    max_path,max_score = p+n, score # 更新路径
            paths[max_path] = max_score # 储存到当前时刻所有节点的最优路径
#     print(paths)
    return max_in_dict(paths)


def cut(s, trans): # 分词函数，也跟前面的HMM基本一致
    if not s: # 空字符直接返回
        return []
    # 字序列转化为id序列。注意，经过我们前面对语料的预处理，字符集是没有空格的，
    # 所以这里简单将空格的id跟句号的id等同起来
    sent_ids = np.array([[vocab2idx.get(c, 0) if c != ' ' else vocab2idx[u'。']
                          for c in s]])
    probas = model.predict(sent_ids)[0] # 模型预测
    nodes = [dict(zip('SBME', i)) for i in probas[:, :4]] # 只取前4个
    nodes[0] = {i:j for i,j in nodes[0].items() if i in 'BS'} # 首字标签只能是b或s
    nodes[-1] = {i:j for i,j in nodes[-1].items() if i in 'ES'} # 末字标签只能是e或s
    tags = viterbi(nodes, trans)[0]
    result = [s[0]]
    for i,j in zip(s[1:], tags[1:]):
        if j in 'BS': # 词的开始
            result.append(i)
        else: # 接着原来的词
            result[-1] += i
    return result

In [36]:
t_ = model.get_weights()[-1][:4,:4] # 从训练模型中取出最新得到的转移矩阵

trans = {}
for i in 'SBME':
    for j in 'SBME':
        trans[i+j] = t_[label2idx[i], label2idx[j]]
result = cut("储存到当前时刻所有节点的最优路径.", trans)
print(result)

['储', '存到', '当前', '时刻', '所有', '节点', '的最', '优路', '径.']


In [34]:
t_ = model.get_weights()[-1][:4,:4] 
print(t_)

[[ 0.2551752   1.3712068  -1.969512   -0.94762987]
 [-1.3502188  -0.11775374  0.6775522   2.1938992 ]
 [-2.6458318  -0.72990716  1.816428    0.97933584]
 [ 0.73843807  1.0811784  -1.9546679  -0.7160042 ]]


In [9]:
import numpy as np
 
class mCRF(object):
    '''实现条件随机场预测问题的维特比算法
    '''
    def __init__(self, V, VW, E, EW):
        '''
        :param V:是定义在节点上的特征函数，称为状态特征
        :param VW:是V对应的权值
        :param E:是定义在边上的特征函数，称为转移特征
        :param EW:是E对应的权值
        '''
        self.V  = V  #点分布表
        self.VW = VW #点权值表
        self.E  = E  #边分布表
        self.EW = EW #边权值表
        self.D  = [] #Delta表，最大非规范化概率的局部状态路径概率
        self.P  = [] #Psi表，当前状态和最优前导状态的索引表s
        self.BP = [] #BestPath，最优路径
        return 
        
    def Viterbi(self):
        '''
        条件随机场预测问题的维特比算法，此算法一定要结合CRF参数化形式对应的状态路径图来理解，更容易理解.
        '''
        self.D = np.full(shape=(np.shape(self.V)), fill_value=.0)
        self.P = np.full(shape=(np.shape(self.V)), fill_value=.0)
        for i in range(np.shape(self.V)[0]):
            #初始化
            if 0 == i:
                self.D[i] = np.multiply(self.V[i], self.VW[i])
                self.P[i] = np.array([0,0,0,0])
            #递推求解布局最优状态路径
            else:
                for y in range(np.shape(self.V)[1]): #delta[i][y=1,2...]
                    for l in range(np.shape(self.V)[1]): #V[i-1][l=1,2...]
                        #前导状态的最优状态路径的概率 + 前导状态到当前状体的转移概率 + 当前状态的概率
                        delta = self.D[i-1, l] + self.E[i-1][l,y]*self.EW[i-1][l,y]  + self.V[i,y]*self.VW[i,y]            #
                        if 0 == l or delta > self.D[i, y]:
                            self.D[i, y], self.P[i, y] = delta, l        
        #返回，得到所有的最优前导状态
        N = np.shape(self.V)[0]
        self.BP = np.full(shape=(N,), fill_value=0.0)
        t_range = -1 * np.array(sorted(-1*np.arange(N)))
        for t in t_range:
            if N-1 == t:#得到最优状态
                self.BP[t] = np.argmax(self.D[-1])
            else: #得到最优前导状态
                self.BP[t] = self.P[t+1, int(self.BP[t+1])]
        
        #最优状态路径表现在存储的是状态的下标，我们执行存储值+1转换成示例中的状态值
        #也可以不用转换，只要你能理解，self.BP中存储的0是状态1就可以~~~~
        self.BP += 1
        return self.BP


In [21]:
s = "最优状态路径表现在存储的是状态的下标，我们执行存储值+1转换成示例中的状态值"
sent_ids = np.array([[vocab2idx[c] if c != ' ' else vocab2idx[u'。']
                          for c in s]])
S = np.array(model.predict(sent_ids)[0][:, :4])  
print(S)
SW = S
crf_tran = [[ 0.2551752,   1.3712068,  -1.969512,   -0.94762987],
             [-1.3502188,  -0.11775374,  0.6775522,   2.1938992 ],
             [-2.6458318,  -0.72990716,  1.816428,    0.97933584],
             [ 0.73843807,  1.0811784,  -1.9546679,  -0.7160042 ]]
def_tran = [[0.5,0.5,0,0],[0,0,0.5,0.5],[0,0,0.5,0.5],[0.5,0.5,0,0]]
E = np.array([def_tran for i in s])
EW = E

crf = mCRF(S, SW, E, EW)
ret = crf.Viterbi()
print('最优状态路径为:', ret)
# SBME
res_str = ''
for i in range(len(s)):
    if ret[i] == 1:
        res_str+=s[i]+' '
    elif ret[i] == 4:
        res_str+=s[i]+" "
    else:
        res_str+=s[i]
print(res_str)
    

[[8.65688682e-01 3.72400582e-02 1.82862226e-02 7.87850767e-02]
 [4.15409356e-01 5.15539765e-01 9.92219336e-03 5.91286980e-02]
 [4.26491676e-03 6.06373310e-01 3.53826255e-01 3.55355330e-02]
 [6.59347838e-03 2.97479751e-03 1.18541606e-02 9.78577614e-01]
 [6.44410634e-03 9.66963649e-01 2.25561112e-02 4.03611222e-03]
 [5.04374411e-03 7.52461892e-06 9.66842577e-04 9.93981898e-01]
 [2.51069311e-02 9.64875996e-01 1.92051532e-03 8.09657108e-03]
 [9.83085833e-04 9.02118012e-02 1.30521273e-02 8.95752907e-01]
 [6.75192535e-01 7.76444969e-04 1.19561227e-02 3.12074870e-01]
 [1.63667300e-03 9.92717028e-01 5.59417391e-03 5.20654394e-05]
 [1.17504084e-03 1.01234968e-06 1.14874201e-04 9.98709083e-01]
 [9.96805310e-01 3.03528830e-03 1.36517512e-04 2.29055586e-05]
 [9.85252440e-01 5.69225755e-04 1.32773785e-04 1.40456446e-02]
 [2.74922210e-03 9.95136559e-01 1.95294677e-03 1.61218166e-04]
 [2.34851008e-03 1.07490109e-06 2.55642753e-05 9.97624815e-01]
 [9.99937773e-01 4.06726322e-05 1.75557707e-05 3.995579

[[ 1.609243   -0.60307544 -0.5321975  -1.350059  ]
 [ 2.1912084  -0.50220716 -1.9891578  -1.5365913 ]
 [-1.2706817   0.44745424  0.21643111 -0.7846817 ]
 [-0.79834855 -2.2858722  -0.7888813   2.3905082 ]
 [-0.4829742   1.3791542  -0.72924024 -1.4593649 ]
 [-0.21598284 -4.392511   -1.1265367   3.3144002 ]
 [-2.0698733   2.1803555  -0.8679491  -0.8824459 ]
 [-1.6855692  -1.5468001  -1.2390535   2.8636303 ]
 [ 3.8489604  -4.0107965  -0.8362013  -1.6430526 ]
 [ 0.26872087  1.424837   -0.645475   -3.6561563 ]
 [ 0.7524922  -3.4886906  -1.6522187   2.5781956 ]
 [ 4.5253954  -1.041991    0.11846626 -5.8860846 ]
 [ 4.114442   -5.469349   -1.738103   -0.07103527]
 [-1.6244731   2.3750694  -0.2490682  -3.1855345 ]
 [ 1.146612   -5.647822   -2.6322265   4.5583663 ]
 [ 5.8469195  -1.4046049   0.5016388  -8.909219  ]
 [ 2.5169191   1.220266   -2.5986495  -2.5697556 ]
 [ 0.16200602 -1.077709   -0.27034223  0.06126173]]
最优状态路径为: [1. 3. 1. 3. 4. 2. 1. 4. 2. 4. 2. 4. 2. 4. 2. 4. 1. 3.]


[[ 0.2551752  -1.3502188  -2.6458318   0.73843807]
 [ 1.3712068  -0.11775374 -0.72990716  1.0811784 ]
 [-1.969512    0.6775522   1.816428   -1.9546679 ]
 [-0.94762987  2.1938992   0.97933584 -0.7160042 ]]
